<a href="https://colab.research.google.com/github/DiegoJustoAgeitos/intro_deep_learning/blob/main/Diego_Justo_Ageitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en preguntas de código y preguntas teóricas, con la puntuación que se indica a continuación. La puntuación máxima será 10.

- [Actividad 1: Redes Densas](#actividad_1): 10 pts
    - Correcta normalización: máximo de 0.5 pts
    - [Cuestión 1](#1.1): 1.5 pts
    - [Cuestión 2](#1.2): 1.5 pts
    - [Cuestión 3](#1.3): 1.5 pts
    - [Cuestión 4](#1.4): 1 pts
    - [Cuestión 5](#1.5): 1 pts
    - [Cuestión 6](#1.6): 1 pts
    - [Cuestión 7](#1.7): 1 pts
    - [Cuestión 8](#1.8): 1 pts


In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

<a name='actividad_1'></a>
# Actividad 1: Redes Densas

Para esta actividad vamos a utilizar el [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality). Con el que trataremos de predecir la calidad del vino.

La calidad del vino puede tomar valores decimales (por ejemplo 7.25), independientemente de que en el dataset de entrenamiento sean números enteros. Por lo tanto, el problema es una `regresión`.

**Puntuación**:

Normalizar las features correctamente (x_train, x_test): 0.5 pts

- Correcta normalización: máximo de 0.5 pts
- [Cuestión 1](#1.1): 1 pt
- [Cuestión 2](#1.2): 1 pt
- [Cuestión 3](#1.3): 0.5 pts
- [Cuestión 4](#1.4): 0.5 pts
- [Cuestión 5](#1.5): 0.5 pts
- [Cuestión 6](#1.6): 0.5 pts
- [Cuestión 7](#1.7): 0.5 pts
- [Cuestión 8](#1.8): 0.5 pts



In [31]:
# Descargar los datos con pandas
df_red = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
    sep=';'
)
df_white = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
    sep=';'
)
df = pd.concat([df_red, df_white])

df.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [32]:
feature_names = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
]


# separar features y target
y = df.pop('quality').values
X = df.copy().values

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)
print('Some qualities: ', y_train[:5])

x_train, y_train shapes: (4872, 11) (4872,)
x_test, y_test shapes: (1625, 11) (1625,)
Some qualities:  [6 7 8 5 6]


In [34]:
## Si quiere, puede normalizar las features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [35]:
print("Media de x_train después de normalizar (debería estar cerca de 0):", x_train.mean(axis=0))
print("Desviación estándar de x_train (debería estar cerca de 1):", x_train.std(axis=0))


Media de x_train después de normalizar (debería estar cerca de 0): [ 1.15064863e-15  1.43563322e-15  2.53269400e-14  3.71760641e-15
 -3.57372975e-16 -1.09096731e-16 -5.61862407e-18  1.85223021e-12
 -3.61682951e-14  5.10059975e-15 -4.98973924e-14]
Desviación estándar de x_train (debería estar cerca de 1): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


<a name='1.1'></a>
## Cuestión 1: Cree un modelo secuencial que contenga 4 capas ocultas(hidden layers), con más de 60 neuronas  por capa, sin regularización y obtenga los resultados.

Puntuación:
- Obtener el modelo correcto: 0.8 pts
- Compilar el modelo: 0.1pts
- Acertar con la función de pérdida: 0.1 pts

In [36]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))  # Salida para regresión (calidad del vino)
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,313 (52.00 KB)

 Trainable params: 13,313 (52.00 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Compilación del modelo
# Código aquí
model.compile(
    optimizer='adam',
    loss='mse',           # Problema de regresión
    metrics=['mae']       # Métrica extra para interpretación
)


In [38]:
# No modifique el código
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 11.9407 - mae: 2.7951 - val_loss: 1.7758 - val_mae: 0.9588
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2981 - mae: 0.8881 - val_loss: 1.0138 - val_mae: 0.7510
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8175 - mae: 0.7106 - val_loss: 0.6969 - val_mae: 0.6236
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6234 - mae: 0.6152 - val_loss: 0.5904 - val_mae: 0.5724
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5300 - mae: 0.5646 - val_loss: 0.5453 - val_mae: 0.5589
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5068 - mae: 0.5526 - val_loss: 0.5529 - val_mae: 0.5595
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4744 - mae: 0.5318 - val_loss: 0.5241 - val_mae: 0.5465
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4734 - mae: 0.5334 - val_loss: 0.5551 - val_mae: 0.5738
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms

In [39]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6741 - mae: 0.5953
Test Loss: [0.7128514051437378, 0.6113939881324768]


<a name='1.2'></a>
## Cuestión 2: Utilice el mismo modelo de la cuestión anterior pero añadiendo al menos dos técnicas distinas de regularización. No es necesario reducir el test loss.

Ejemplos de regularización: [Prevent_Overfitting.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/Prevent_Overfitting.ipynb)

In [40]:
model = tf.keras.models.Sequential()
# Capa 1: regularización L2 + Dropout
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dropout(0.3))
# Capa 2: regularización L1 + Dropout
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model.add(tf.keras.layers.Dropout(0.3))
# Capa 3: regularización combinada L1 y L2 + Dropout
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
model.add(tf.keras.layers.Dropout(0.3))
# Capa 4: regularización L2
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
# Capa de salida
model.add(tf.keras.layers.Dense(1))
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,313 (52.00 KB)

 Trainable params: 13,313 (52.00 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# Compilación del modelo
# Código aquí
# Compilación del modelo
# Código aquí
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)


In [42]:
batch_size=32

In [43]:
# No modifique el código
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=batch_size,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 22.1849 - mae: 3.2249 - val_loss: 8.9606 - val_mae: 1.2038
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.8126 - mae: 1.2230 - val_loss: 6.9640 - val_mae: 0.9342
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 6.8451 - mae: 0.9603 - val_loss: 5.5394 - val_mae: 0.7485
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.4462 - mae: 0.8014 - val_loss: 4.3677 - val_mae: 0.5824
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.5063 - mae: 0.7385 - val_loss: 3.6152 - val_mae: 0.5823
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.7249 - mae: 0.7111 - val_loss: 3.0140 - val_mae: 0.5862
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.0850 - mae: 0.6879 - val_loss: 2.5078 - val_mae: 0.5891
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.5751 - mae: 0.6721 - val_loss: 2.0908 - val_mae: 0.5960
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms

In [44]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6274 - mae: 0.6018
Test Loss: [0.6406022906303406, 0.610497772693634]


<a name='1.3'></a>
## Cuestión 3: Utilice el mismo modelo de la cuestión anterior pero añadiendo un callback de early stopping. No es necesario reducir el test loss.

In [45]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],),kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(1))
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,313 (52.00 KB)

 Trainable params: 13,313 (52.00 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
# Compilación del modelo
# Código aquí
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)


In [47]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,           # Paciencia: número de épocas sin mejorar para parar el entrenamiento
    restore_best_weights=True)
# Código aquí
...
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[early_stop])

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 20.9121 - mae: 3.0536 - val_loss: 9.4890 - val_mae: 1.3423
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.1762 - mae: 1.2973 - val_loss: 7.0765 - val_mae: 0.9617
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.0462 - mae: 1.0196 - val_loss: 5.6461 - val_mae: 0.7920
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.4946 - mae: 0.8054 - val_loss: 4.5170 - val_mae: 0.6438
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.5088 - mae: 0.7231 - val_loss: 3.6617 - val_mae: 0.5777
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.8246 - mae: 0.7265 - val_loss: 3.0489 - val_mae: 0.5767
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.1146 - mae: 0.6676 - val_loss: 2.5983 - val_mae: 0.5997
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.6056 - mae: 0.6564 - val_loss: 2.1761 - val_mae: 0.5879
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms

In [48]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6513 - mae: 0.5974
Test Loss: [0.6622916460037231, 0.6077209115028381]


<a name='1.4'></a>
## Cuestión 4: ¿Podría haberse usado otra función de activación de la neurona de salida? En caso afirmativo especifíquela.

Sí, se puede usar otra función de activación en la neurona de salida, pero depende de lo que queramos predecir.
En este caso, como la calidad del vino puede ser cualquier número, lo mejor es no poner activación para que el modelo saque cualquier valor.
Si solo quisiéramos valores entre 0 y 1, podríamos usar sigmoide.
Y si solo quisiéramos valores positivos, también podríamos usar ReLU.
Pero lo habitual para este tipo de problema es dejar la salida sin activación.

<a name='1.5'></a>
## Cuestión 5:  ¿Qué es lo que una neurona calcula?

**a)** Una función de activación seguida de una suma ponderada  de las entradas.

**b)** Una suma ponderada  de las entradas seguida de una función de activación.

**c)** Una función de pérdida, definida sobre el target.

**d)** Ninguna  de las anteriores es correcta


B

<a name='1.6'></a>
## Cuestión 6:  ¿Cuál de estas funciones de activación no debería usarse en una capa oculta (hidden layer)?

**a)** `sigmoid`

**b)** `tanh`

**c)** `relu`

**d)** `linear`


D


<a name='1.7'></a>
## Cuestión 7:  ¿Cuál de estas técnicas es efectiva para combatir el overfitting en una red con varias capas ocultas? Ponga todas las que lo sean.

**a)** Dropout

**b)** Regularización L2.

**c)** Aumentar el tamaño del test set.

**d)** Aumentar el tamaño del validation set.

**e)** Reducir el número de capas de la red.

**f)** Data augmentation.

C,D

<a name='1.8'></a>
## Cuestión 8:  Supongamos que queremos entrenar una red para un problema de clasificación de imágenes con las siguientes clases: {'perro','gato','persona'}. ¿Cuántas neuronas y que función de activación debería tener la capa de salida? ¿Qué función de pérdida (loss function) debería usarse?


Para este problema de clasificación con tres clases (perro, gato, persona):

*   La capa de salida debe tener 3 neuronas, una por cada clase posible.

*   Se usa la función de activación softmax porque convierte la salida en probabilidades que suman 1. De esta forma, la red nos dice qué probabilidad le da a cada clase (por ejemplo, 80% perro, 10% gato, 10% persona).
*   La función de pérdida que se usa en este caso es categorical_crossentropy, que es la adecuada para problemas de clasificación multiclase.


Esto permite que el modelo aprenda a distinguir entre las distintas clases de forma eficiente y nos devuelva la clase más probable para cada imagen.